In [1]:
!pip install seaborn
import torch
import random
import torch.nn as nn 
import pathlib
import numpy as np 

import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
print(tf.__version__)

You should consider upgrading via the 'pip install --upgrade pip' command.


/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

1.14.0


/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorboard/compat/tensorflow_stub/dtypes.py:544: FutureWa

In [9]:
# import os
# os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
# os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [10]:
dataset_path = keras.utils.get_file("auto-mpg.data", "http://archive.ics.uci.edu/ml/machine-learning-databases/auto-mpg/auto-mpg.data")
dataset_path

'/home/coder/.keras/datasets/auto-mpg.data'

In [11]:
# sns.pairplot(dataset[column_names], diag_kind="kde")

In [12]:
def load_dataset(idxs=[2,3]):
    column_names = ['MPG','Cylinders','Displacement','Horsepower','Weight',
                    'Acceleration', 'Model Year', 'Origin']
    raw_dataset = pd.read_csv(dataset_path, names=column_names,
                        na_values = "?", comment='\t',
                        sep=" ", skipinitialspace=True)

    ds = raw_dataset.copy().to_numpy()
    dataset = []

    for i, element in enumerate(ds):
        is_nan = False
        for x in element:
            if np.isnan(x):
                # print('here', i )
                is_nan=True
        
        if is_nan is False:
            dataset.append(element)

    dataset = np.array(dataset)
    x = dataset[:, idxs]
    y = dataset[:, 0]
    return x, y

def norm(x, x_mean=None, x_std=None):
    if x_mean is None:
        x_mean = torch.mean(x, dim=0, keepdim=True)
        x_std = torch.std(x, dim=0, keepdim=True)

    x = (x - x_mean) / x_std
    return x, x_mean, x_std

def dnorm(x, x_mean=None, x_std=None):
    output = x * x_std + x_mean
    return output

In [13]:
from tqdm import tqdm
class RegressionNeuron2(nn.Module):
    def __init__(self, in_dim=2, n_hidden=60, learning_rate=1e-2, w1=1.11, w2=0.22):
            '''
            Args:
              in_dim: The dimension of input data  
              n_hidden: The dimension of hidden layer
              learning_rate: The initial learning rate for training a network 
            '''
            super().__init__()

            self.model = nn.Sequential(
                nn.BatchNorm1d(in_dim),
                nn.Linear(in_dim, n_hidden),
                nn.ReLU(), 
                nn.Linear(n_hidden, n_hidden//2),
                nn.Linear(n_hidden//2, 1)
                )

            self.optim = torch.optim.Adam(self.model.parameters(), betas=(0.9, 0.999), eps=1e-8, lr=learning_rate)
            self.mse_loss = nn.MSELoss()
            self.l1_loss = nn.L1Loss()
            self.bce = nn.BCEWithLogitsLoss()
            self.w1 = w1
            self.w2 = w2
    
    def _calculate_loss(self, x, y):
        loss =  self.w1*self.mse_loss(x, y) + self.w2*self.l1_loss(x,y)
        return loss 

    def forward(self, x):
        z = self.model(x)
        return z

    def train(self, x, y, epochs=1000, batch_size=20):
        B = batch_size
        n = len(y)
        for i in tqdm(range(epochs)):
            
            for j in range(0, n, B):
                if j+B > n:
                    m = n-j
                    continue
                else:
                    m=B
                 
                xi = [v.unsqueeze(0) for v in x[j:(j+m)]]
                yi = y[j:(j+m)]
                
                xi = torch.cat(xi, dim=0)
                
                pred_y = self.forward(xi)#.reshape(-1)
                pred_y = pred_y.reshape(-1)
                # print(pred_y.shape, yi.shape)
                loss = self._calculate_loss(pred_y, yi)
                self.model.zero_grad()
                self.optim.zero_grad()
                loss.backward()
                self.optim.step()

            if (i+1) % 10 == 0:  # 여기 나누는 값을 조정해서 프린트하는 횟수를 조절
                ppred = self.predict(x_eval_torch)
                eval_loss = np.mean((ppred - y_eval)**2)
                print("epoch {} : err = {:.4f}, eval err = {:.4f}".format(i + 1, loss, eval_loss))

    def predict(self, x):
        x, _, _= norm(x, x_train_mean, x_train_std)
        output = self.forward(x)
        output = dnorm(output, y_train_mean, y_train_std)
        output = output.data.cpu().numpy()
        return output

In [14]:
x, y = load_dataset(idxs=[2,3])
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)  # 수정 금지 
x_train, x_eval, y_train, y_eval = train_test_split(x_train, y_train, test_size=0.125, random_state=999)
x_train_torch = torch.FloatTensor(x_train)#.cuda()
y_train_torch = torch.FloatTensor(y_train)#.cuda()
x_eval_torch = torch.FloatTensor(x_eval)#.cuda()
y_eval_torch = torch.FloatTensor(y_eval)#.cuda()
x_test_torch = torch.FloatTensor(x_test)#.cuda()
y_test_torch = torch.FloatTensor(y_test)#.cuda()
x_train_norm, x_train_mean, x_train_std = norm(x_train_torch)
y_train_norm, y_train_mean, y_train_std = norm(y_train_torch)


In [15]:

random_seed = 222
np.random.seed(random_seed)
torch.manual_seed(random_seed)
random.seed(random_seed)

neuron = RegressionNeuron2(in_dim=x_train_norm.shape[-1], n_hidden=32,  learning_rate=1,
w1=1.15, w2=0.185,
)#.cuda()
neuron.train(x_train_norm, y_train_norm, epochs=30, batch_size=64)

output = np.mean((neuron.predict(x_test_torch) - y_test)**2)
print(output)

100%|██████████| 30/30 [00:00<00:00, 122.72it/s]

epoch 10 : err = 82.7303, eval err = 3093.1055
epoch 20 : err = 1.8106, eval err = 81.6620
epoch 30 : err = 1.1924, eval err = 61.0533
51.040593618875086
